In [1]:
# author: Yuman Lin
# July 24 2020

In [2]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)
X_train = mnist["data"][:60000]
X_test  = mnist["data"][60000:]
y_train = mnist["target"][:60000]
y_test  = mnist["target"][60000:]

In [3]:
import timeit

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
start = timeit.default_timer()
X_reduced = pca.fit_transform(X_train)
stop = timeit.default_timer()
print("'time of PCA library: ", stop - start)

'time of PCA library:  12.062346611000066


In [7]:
X_reduced.shape

(60000, 154)

In [8]:
X_reduced

array([[ 123.93258866, -312.67426202,  -24.51405176, ...,   55.01899792,
         -20.08327427,   39.58995229],
       [1011.71837587, -294.85703827,  596.33956104, ...,    7.24129874,
         -12.45780869,  -12.7432306 ],
       [ -51.84960805,  392.17315286, -188.50974943, ...,  -54.19582221,
          48.47979747,  -73.27826256],
       ...,
       [-178.0534496 ,  160.07821109, -257.61308227, ...,   55.54485537,
          87.99883556,   -5.78979735],
       [ 130.60607208,   -5.59193642,  513.85867395, ...,   23.30835402,
           5.06237836,  -65.26525587],
       [-173.43595244,  -24.71880226,  556.01889393, ...,   52.4956069 ,
          12.63192292,  -45.74001227]])

In [9]:
type(X_train)

numpy.ndarray

In [10]:
def pca(dataMat, topNfeat=0.95):

    # featurenormalize
    meanVals = np.mean(dataMat, axis=0)
    meanRemoved = dataMat - meanVals

    # computeCov
    covmat = np.cov(meanRemoved, rowvar=0)
    print(covmat)
    print(type(covmat))

    # computeEig
    eigVals, eigVects = np.linalg.eig(np.mat(covmat))

    # computeDim
    if topNfeat < 1:
      sum = 0
      eigValInd = np.argsort(eigVals)
      eigValsort = np.sort(eigVals)
      
      for i in range((len(eigValsort)-1), 0, -1):
        sum += eigValsort[i]
        dim = i
        if (sum / eigValsort.sum() >= topNfeat):
          break
      topNfeat = dataMat.shape[1] - dim
    
    eigValInd = np.argsort(eigVals)
    eigValInd = eigValInd[:-(topNfeat+1):-1]    
    redEigVects = eigVects[:, eigValInd]        

   
    lowDDataMat = meanRemoved * redEigVects     
    reconMat = (lowDDataMat * redEigVects.T) + meanVals 
    return np.array(lowDDataMat), np.array(reconMat)


lowDDataMat,reconMat = pca(X_train, 0.95)
print(lowDDataMat.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
(60000, 154)


In [11]:
start = timeit.default_timer()
lowDDataMat, reconMat = pca(X_train, 0.95)
end = timeit.default_timer()
print("time of Yuman's PCA: ", end - start)
print(lowDDataMat.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
time of Yuman's PCA:  3.7113505499999064
(60000, 154)


In [13]:
lowDDataMat

array([[ -123.93258866,  -312.67426202,   -24.51405176, ...,
          -55.01899792,    20.08327427,   -39.58995229],
       [-1011.71837587,  -294.85703827,   596.33956104, ...,
           -7.24129874,    12.45780869,    12.7432306 ],
       [   51.84960805,   392.17315286,  -188.50974943, ...,
           54.19582221,   -48.47979747,    73.27826256],
       ...,
       [  178.0534496 ,   160.07821109,  -257.61308227, ...,
          -55.54485537,   -87.99883556,     5.78979735],
       [ -130.60607208,    -5.59193642,   513.85867395, ...,
          -23.30835402,    -5.06237836,    65.26525587],
       [  173.43595244,   -24.71880226,   556.01889393, ...,
          -52.4956069 ,   -12.63192292,    45.74001227]])

In [12]:
import cupy as cp

In [14]:
X_train_cp = cp.array(X_train)
type(X_train_cp)

cupy.core.core.ndarray

In [15]:
def pca_cp_v2(dataMat, topNfeat=999999):

    
    meanVals = cp.mean(dataMat, axis=0)
    meanRemoved = dataMat - meanVals

    covmat = cp.cov(meanRemoved, rowvar=0)
    # print(covmat)
    #covmat = cp.asnumpy(covmat)

    eigVals, eigVects = cp.linalg.eigh(covmat)

    # computeDim
    if topNfeat < 1:
      sum = 0
      eigValsort = cp.sort(eigVals)
      
      for i in range((len(eigValsort)-1), 0, -1):
        sum += eigValsort[i]
        dim = i
        if (sum / eigValsort.sum() >= topNfeat):
          break
      topNfeat = dataMat.shape[1] - dim
    
    eigValInd = cp.argsort(eigVals)
    eigValInd = eigValInd[:-(topNfeat+1):-1]    
    redEigVects = eigVects[:, eigValInd]    
    print(type(meanRemoved))  
    print(type(redEigVects))  

   
    lowDDataMat = meanRemoved.dot(redEigVects)     
    reconMat = (lowDDataMat.dot(redEigVects.T)) + meanVals 
    return cp.array(lowDDataMat), cp.array(reconMat)

In [17]:
#%time lowDDataMat, reconMat = pca_cp_v2(X_train_cp, 3)
start = timeit.default_timer()
lowDDataMat, reconMat = pca_cp_v2(X_train_cp, 0.95)
end = timeit.default_timer()
print("time of cupy PCA v2: ", end - start)

<class 'cupy.core.core.ndarray'>
<class 'cupy.core.core.ndarray'>
time of cupy PCA v2:  0.3905913819999114
